# Trade System
- [x] [Get Binance Data](#Get-Data-from-Binance)
- [ ] [Database](#Get-Data)
- [ ] [Strategy](#Strategy)
- [ ] [Backtesting](#Backtesting)
- [ ] [Optimization](#Optimization)
- [ ] [Send Order](#Send-Order)

### API parameter
    https://binance-docs.github.io/apidocs/futures/cn/#api
### OrderTypes
    LIMIT, MARKET, STOP, STOP_MARKET, TAKE_PROFIT, TAKE_PROFIT_MARKET, TRAILING_STOP_MARKET
### Side
    BUY, SELL
### Interval
    1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
### Exchange information
    https://api.binance.com/api/v1/exchangeInfo
### Trades (from ID)
    /fapi/v1/trades  
    /fapi/v1/historicalTrades
### K lines
    /fapi/v1/klines  
    https://binance-docs.github.io/apidocs/futures/cn/#k  
    https://binance-docs.github.io/apidocs/futures/cn/#k-6  
|name	|type	|require |discription |
| ---   | ---   | ---    | ---        |
|symbol|STRING	|YES |交易對|
|interval |ENUM	|YES |時間間隔|
|startTime |LONG |NO |起始時間|
|endTime |LONG |NO |結束時間|
|limit |INT |NO |筆數 default:500 max:1500| 


In [7]:
import requests
import numpy as np
import pandas as pd
from enum import Enum
from datetime import datetime

## Get Data from Binance

In [8]:
class Interval(Enum):
    """
    Interval for klines
    
    """
    Minute_1 = '1m'
    Minute_3 = '3m'
    Minute_5 = '5m'
    Minute_15 = '15m'
    Minute_30 = '30m'
    Hour_1 = '1h'
    Hour_2 = '2h'
    Hour_4 = '4h'
    Hour_6 = '6h'
    Hour_8 = '8h'
    Hour_12 = '12h'
    Day_1 = '1d'
    Day_3 = '3d'
    Week_1 = '1w'
    Month_1 = '1M'
    
class OrderType(Enum):
    """
    Order type
    
    """
    LIMIT = "LIMIT"
    MARKET = "MARKET"
    STOP = "STOP"
    TAKE_PROFIT = "TAKE_PROFIT"
    STOP_MARKET = "STOP_MARKET"
    TAKE_PROFIT_MARKET = "TAKE_PROFIT_MARKET"
    TRAILING_STOP_MARKET = "TRAILING_STOP_MARKET"


class positionside(Enum):
    BOTH = 'BOTH'
    LONG = 'LONG'
    SHORT = 'SHORT'

class TimeInForce(Enum):
    GTC="GTC"
    IOD="IOC"
    FOK="FOK"
    GTX="GTX"

class OrderSide(Enum):
    """
    Order Side
    
    """
    BUY = "BUY"
    SELL = "SELL"

class contractType(Enum):
    PERPETUAL = "PERPETUAL"
    CURRENT_MONTH = "CURRENT_MONTH"
    NEXT_MONTH = "NEXT_MONTH"
    CURRENT_QUARTER = "CURRENT_QUARTER"
    NEXT_QUARTER = "NEXT_QUARTER"

In [9]:
def get_exchange_info():
    """
    All Coin List
    
    """
    return requests.get("https://api.binance.com/api/v1/exchangeInfo").json()

In [10]:
def get_symbol(target: str = None, margined = "USDT", status = "TRADING"):
    """
    Get Specific Symbol List
    
    """
    SymbolList = get_exchange_info()
    SymbolList = [s["symbol"] for s in SymbolList["symbols"] if s["status"] == status]
    if target:
        SymbolList = [s for s in SymbolList if s[:len(target)] == target]
    if margined:
        SymbolList = [s for s in SymbolList if s[-len(margined):] == margined]
    return SymbolList

In [11]:
def datetime_timestamp(year, month, day, hour = 0, min = 0, second = 0):
    """
    Milliseconds Timestamp | UTC +8
    """
    time = datetime(year, month, day, hour, min, second).timestamp()
    return int(time) * 1000

In [12]:
def get_klines(symbol: str, interval, starttime=None, endtime=None, limit=1000):
    """
    Get Historical Klines from Binance
    
    Parameters
    ----------
    symbol : list of str
    interval : str
        Time scale can be 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    starttime : int
        Timestamp
        
    Returns
    -------
    out : list
    
    """
    params = {"symbol": symbol,
              "interval": interval,
              "limit": limit}
    if starttime:
        params["startTime"] = starttime
    if endtime:
        params["endTime"] = endtime
    url = "https://api.binance.com/api/v3/klines"
    
    data = []
    
    while starttime <= endtime:
        parameter = "&".join(f"{key}={params[key]}" for key in params.keys())
        path = url + "?" + parameter
        kline = requests.get(path).json()
        print(datetime.fromtimestamp(kline[0][0] / 1000), datetime.fromtimestamp(kline[-1][0] / 1000), len(kline))
        data += kline
        starttime = kline[-1][0] + 1
        params["startTime"] = starttime
        
    print("------", symbol, "------", datetime.fromtimestamp(data[0][0] / 1000), "-", datetime.fromtimestamp(data[-1][0] / 1000))    

    return data  

In [13]:
symbol_list = ["BTCUSDT"]
# symbol_list = get_symbol("BTC")
interval = Interval.Minute_30.value
start_time = datetime_timestamp(2021, 10, 1, 0, 0, 0)
end_time = datetime_timestamp(2022, 6, 30, 0, 0, 0)
column_name = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume", 
       "Number of trades", "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"]
Data = pd.DataFrame(get_klines("BTCUSDT", interval, start_time, end_time), columns = column_name)

2021-10-01 00:00:00 2021-10-21 19:30:00 1000
2021-10-21 20:00:00 2021-11-11 15:30:00 1000
2021-11-11 16:00:00 2021-12-02 11:30:00 1000
2021-12-02 12:00:00 2021-12-23 07:30:00 1000
2021-12-23 08:00:00 2022-01-13 03:30:00 1000
2022-01-13 04:00:00 2022-02-02 23:30:00 1000
2022-02-03 00:00:00 2022-02-23 19:30:00 1000
2022-02-23 20:00:00 2022-03-16 15:30:00 1000
2022-03-16 16:00:00 2022-04-06 11:30:00 1000
2022-04-06 12:00:00 2022-04-27 07:30:00 1000
2022-04-27 08:00:00 2022-05-18 03:30:00 1000
2022-05-18 04:00:00 2022-06-07 23:30:00 1000
2022-06-08 00:00:00 2022-06-28 19:30:00 1000
2022-06-28 20:00:00 2022-06-30 00:00:00 57
------ BTCUSDT ------ 2021-10-01 00:00:00 - 2022-06-30 00:00:00


## Get Data

In [14]:
col = ["Open time", "Open", "High", "Low", "Close", "Volume"]
df = Data[col]
df = df.rename({"Open time": 'Date'}, axis=1)
df.Date = (df["Date"] / 1000).apply(datetime.fromtimestamp)
df = df.astype({col: float for col in df.columns[1:]})

In [15]:
df

,Date,Open,High,Low,Close,Volume
0,2021-10-01 00:00:00,43149.02,43218.00,42914.33,42954.29,801.52103
1,2021-10-01 00:30:00,42954.30,42997.77,42819.86,42950.67,800.71902
2,2021-10-01 01:00:00,42950.66,43314.50,42933.29,43017.60,864.06192
3,2021-10-01 01:30:00,43017.61,43400.00,42906.69,43333.12,1128.33387
4,2021-10-01 02:00:00,43333.12,43473.45,43261.84,43397.54,616.15522
...,...,...,...,...,...,...
13052,2022-06-29 22:00:00,20083.99,20177.31,20036.71,20118.56,2329.46133
13053,2022-06-29 22:30:00,20118.56,20254.20,20050.10,20126.02,1903.32274
13054,2022-06-29 23:00:00,20126.01,20142.79,20000.00,20064.34,2048.20718
13055,2022-06-29 23:30:00,20064.35,20124.71,20030.01,20084.48,1112.31012


## Strategy

```
def strategy_signal(data, para1, para2...):    
    return array
```

In [84]:
pd.__version__

'1.4.3'

In [ ]:
df['MA5'] = pd.rolling_mean(df['Close'], 5)
df['MA10'] = pd.rolling_mean(df['Close'], 10)
df['MA20'] = pd.rolling_mean(df['Close'], 20)
df['RSV'] = (df['Close'] - pd.rolling_min(df['Low'], 9)) / (pd.rolling_max(df['High'], 9) - pd.rolling_min(df['Low'], 9))
df['K'] = pd.ewma(df['RSV'], adjust=False, alpha=1/3)
df['D'] = pd.ewma(df['K'], adjust=False, alpha=1/3)
TR = pd.concat([df[['High', 'Low']], df['Close'].shift(1)], axis=1, ignore_index=True)
df['TR'] = TR.max(axis=1) - TR.min(axis=1)
df['ATR'] = pd.ewma(df['TR'], adjust=False, alpha=1/15)
df['TRMA'] = pd.rolling_mean(df['TR'], 15)
df['DM_P'] = np.where((df['High'] - df['High'].shift(1)) > 0, df['High'] - df['High'].shift(1), 0)
df['DM_N'] = np.where((df['Low'] - df['Low'].shift(1)) < 0, df['Low'].shift(1) - df['Low'], 0)
df['ADM_P'] = pd.ewma(df['DM_P'], adjust=False, alpha=1/15)
df['ADM_N'] = pd.ewma(df['DM_N'], adjust=False, alpha=1/15)
df['DX'] = ((df['ADM_P'] - df['ADM_N']) / (df['ADM_P'] + df['ADM_N'])).abs() * 100
df['ADX'] = pd.ewma(df['DX'], adjust=False, alpha=1/15)

In [205]:
def MA(close, num = 20):
    """
    Moving Average
    
    Parameters
    ----------
    close: price list
    num: lenth
    
    """
    return close.rolling(num).mean()

def EMA(close, num = 20, adjust = False):
    """
    Exponential Moving Average
    
    Parameters
    ----------
    close: price list
    num: lenth
    
    """
    return close.ewm(span = num, adjust = adjust)
    
def WMA(close, num = 20, adjust = False):
    """
    Weighted Moving Average
    
    Parameters
    ----------
    close: price list
    num: lenth
    
    """    
    weight = np.arange(1, num + 1) / (0.5 * num * (num + 1))
    dot = lambda x: np.dot(weight, x)
    close_r = close.rolling(num)
    return close_r.apply(dot)
    
def KD(high, low, close, num = 9, ratio = 1/3, num_slow = 10):
    """
    Stochastic Oscillator
    
    Parameters
    ----------
    high, low, close: price list
    num: lenth
    ratio: 
    num_slow: 
    
    """
    RSV = (close - low.rolling(num).min()) / (high.rolling(num).max() - low.rolling(num).min())
    K = RSV.ewm(alpha = ratio, adjust = False)
    D = K.ewm(alpha = ratio, adjust = False)
    return K, D

    

In [59]:
a = df.copy()

In [159]:
a = pd.DataFrame({'B': [0, 1, 2, 3, 5]})
a

,B
0,0
1,1
2,2
3,3
4,5


In [183]:
b = np.arange(1, 3)
b

array([1, 2])

In [196]:
linear = lambda x: np.dot(x, b)

## Backtesting

```
def strategy_performance(symbol, interval, starttime, endtime, strategy, para..., output=[drawdown, profit...]):
    return summary[output], graph 
```

## Optimization

```
def strategy_optimization(symbol, interval, starttime, endtime, strategy, para..., 
                          output=[drawdown, profit...], method[exhaustive, genetic, bp]):
    return para, output
```

## Send Order

傳送下單資訊，收取下單回報，追蹤order狀況